In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans

In [2]:
nCluster=8
df = pd.read_pickle("data/dataframe")
yearsString = ["14", "15", "16", "17", "18", "19", "20"]
arbevetel = ["TAH197"]
koltseg = ["TAC008",
"TAC009","TAC011","TAC010","TAC078","TAH014","TAH015","TAC012","TAC013","TAC014","TAC015","TAC016","TAC018","TAC063","TAH021","TAC109","TAC190","TAC059","TAB024","TAH197"]
merlegfoosszeg=["TAH186",
"TAH230","TAH033","TAH087","TAH088","TAH041","TAH042","TAH231","TAH043","TAH044","TAH232","TAH045", "TAH187", "TAI001",
"TAI002","TAI019","TAI020","TAI022","TAI023","TAI041","TAI042","TAI043","TAI044","TAI033","TAH034","TAI045","TAI046","TAI035","TAI047","TAI048","TAI036","TAI049","TAI050","TAI037","TAI051","TAI052","TAI053","TAI054","TAI055","TAI056","TAI057","TAH190","TAI038","TAI058", "TAB024","TAH197"]
liabilities = ["TAH209","TAH051","TAH227","TAH054","TAH055","TAH180","TAH199"]
toke = ["TAH002","TAH003","TAH004","TAH005","TAH011","TAH007","TAH233","TAH234","TAH012","TAH048","TAH189","TAH208","TAH060","TAH179","TAH187"]

ratios = pd.DataFrame()
for y in yearsString:
    for i in arbevetel:
        ratios[y+i+"arbev"] = df[y+i]/df[y+"TAC002"]
    for i in merlegfoosszeg:
        ratios[y+i+"mf"] = df[y+i]/df[y+"TAH061"]
    for i in liabilities:
        ratios[y+i+"liab"] = df[y+i]/(df[y+"TAH051"]+df[y+"TAH054"])
    for i in toke:
        ratios[y+i+"toke"] = df[y+i]/(df[y+"TAH001"]+df[y+"TAH012"]+df[y+"TAH048"]+df[y+"TAH189"]+df[y+"TAH208"]+df[y+"TAH060"]+df[y+"TAH179"]+df[y+"TAH187"]+df[y+"TAH059"])
    for i in koltseg:
        ratios[y+i+"kolt"] = df[y+i]/(df[y+"TAC002"]+df[y+"TAC006"]-df[y+"TAC019"])
ratios = ratios.replace(np.inf, 0)
ratios = ratios.replace(-np.inf, 0)        
ratios.fillna(0, inplace=True)
        
cRatios = pd.DataFrame()
for y in range(14, 20):
    for i in arbevetel:
        cRatios[str(y)+"c"+str(y+1)+"arbev"] = (ratios[str(y+1)+i+"arbev"]-ratios[str(y)+i+"arbev"])/ratios[str(y)+i+"arbev"]
    for i in merlegfoosszeg:
        cRatios[str(y)+"c"+str(y+1)+"mf"] = (ratios[str(y+1)+i+"mf"]-ratios[str(y)+i+"mf"])/ratios[str(y)+i+"mf"]
    for i in liabilities:
        cRatios[str(y)+"c"+str(y+1)+"liab"] = (ratios[str(y+1)+i+"liab"]-ratios[str(y)+i+"liab"])/ratios[str(y)+i+"liab"]
    for i in toke:
        cRatios[str(y)+"c"+str(y+1)+"toke"] = (ratios[str(y+1)+i+"toke"]-ratios[str(y)+i+"toke"])/ratios[str(y)+i+"toke"]
    for i in koltseg:
        cRatios[str(y)+"c"+str(y+1)+"kolt"] = (ratios[str(y+1)+i+"kolt"]-ratios[str(y)+i+"kolt"])/ratios[str(y)+i+"kolt"]

cRatios = cRatios.replace(np.inf, 0)
cRatios = cRatios.replace(-np.inf, 0)        
cRatios.fillna(0, inplace=True)

prodInTime = pd.DataFrame()
for i in range(14, 21): 
    prodInTime[str(i)] = df[str(i)+"Productivity"]
prodInTime = TimeSeriesScalerMeanVariance().fit_transform(prodInTime)

timeSeriesClusterModel = TimeSeriesKMeans(n_clusters=nCluster, metric="dtw", max_iter=10, random_state=15)
timeSeriesClusterModel.fit(prodInTime)
targetLabels = timeSeriesClusterModel.predict(prodInTime)

<ipython-input-2-36531ed9f2c0>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratios[y+i+"mf"] = df[y+i]/df[y+"TAH061"]
<ipython-input-2-36531ed9f2c0>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratios[y+i+"liab"] = df[y+i]/(df[y+"TAH051"]+df[y+"TAH054"])
<ipython-input-2-36531ed9f2c0>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fr

In [3]:
ratios.shape

(13869, 623)

In [53]:
for i in range(0, 8):
    estimator = DecisionTreeClassifier(min_samples_split = 150)
    selector = RFE(estimator, n_features_to_select=50, step=0.05, verbose = 4)
    selector = selector.fit(cRatios, targetLabels==i)
    np.save("cRatiosRFEmaskC"+str(i), selector.support_)

Fitting estimator with 534 features.
Fitting estimator with 508 features.
Fitting estimator with 482 features.
Fitting estimator with 456 features.
Fitting estimator with 430 features.
Fitting estimator with 404 features.
Fitting estimator with 378 features.
Fitting estimator with 352 features.
Fitting estimator with 326 features.
Fitting estimator with 300 features.
Fitting estimator with 274 features.
Fitting estimator with 248 features.
Fitting estimator with 222 features.
Fitting estimator with 196 features.
Fitting estimator with 170 features.
Fitting estimator with 144 features.
Fitting estimator with 118 features.
Fitting estimator with 92 features.
Fitting estimator with 66 features.
Fitting estimator with 534 features.
Fitting estimator with 508 features.
Fitting estimator with 482 features.
Fitting estimator with 456 features.
Fitting estimator with 430 features.
Fitting estimator with 404 features.
Fitting estimator with 378 features.
Fitting estimator with 352 features.
Fit

In [47]:
cRatios.columns[selector.support_]

Index(['14c15TAH232mf', '14c15TAI051mf', '14c15TAH197mf', '14c15TAH189toke',
       '14c15TAH187toke', '14c15TAC009kolt', '14c15TAH015kolt',
       '14c15TAC012kolt', '14c15TAC013kolt', '14c15TAC018kolt',
       '15c16TAH004toke', '15c16TAH189toke', '15c16TAH014kolt',
       '15c16TAC012kolt', '15c16TAC013kolt', '16c17TAH042mf', '16c17TAI051mf',
       '16c17TAH004toke', '16c17TAH014kolt', '16c17TAC012kolt',
       '16c17TAC013kolt', '17c18TAH045mf', '17c18TAH187mf', '17c18TAI035mf',
       '17c18TAH004toke', '17c18TAH187toke', '17c18TAC009kolt',
       '17c18TAC013kolt', '18c19TAH187mf', '18c19TAI001mf', '18c19TAH004toke',
       '18c19TAH189toke', '18c19TAH187toke', '18c19TAC008kolt',
       '18c19TAC010kolt', '18c19TAC078kolt', '18c19TAC012kolt',
       '18c19TAC015kolt', '19c20TAH044mf', '19c20TAH187mf', '19c20TAI001mf',
       '19c20TAI057mf', '19c20TAH004toke', '19c20TAH189toke',
       '19c20TAH187toke', '19c20TAC008kolt', '19c20TAC009kolt',
       '19c20TAC012kolt', '19c20TAC01

In [48]:
np.save("cRatiosRFEmask", selector.support_)

In [54]:
for i in range(0, 8):
    estimator = DecisionTreeClassifier(min_samples_split = 150)
    selector = RFE(estimator, n_features_to_select=50, step=0.05, verbose = 4)
    selector = selector.fit(ratios, targetLabels==i)
    np.save("ratiosRFEmaskC"+str(i), selector.support_)

Fitting estimator with 623 features.
Fitting estimator with 592 features.
Fitting estimator with 561 features.
Fitting estimator with 530 features.
Fitting estimator with 499 features.
Fitting estimator with 468 features.
Fitting estimator with 437 features.
Fitting estimator with 406 features.
Fitting estimator with 375 features.
Fitting estimator with 344 features.
Fitting estimator with 313 features.
Fitting estimator with 282 features.
Fitting estimator with 251 features.
Fitting estimator with 220 features.
Fitting estimator with 189 features.
Fitting estimator with 158 features.
Fitting estimator with 127 features.
Fitting estimator with 96 features.
Fitting estimator with 65 features.
Fitting estimator with 623 features.
Fitting estimator with 592 features.
Fitting estimator with 561 features.
Fitting estimator with 530 features.
Fitting estimator with 499 features.
Fitting estimator with 468 features.
Fitting estimator with 437 features.
Fitting estimator with 406 features.
Fit